In [58]:
# %matplotlib qt
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import random


#lorenz system parameters
a, b, c = 10.0, 30.0, 2


#critical points
crit_pnt1 = np.array([np.sqrt(c * (b - 1)), np.sqrt(c * (b - 1)), b - 1])
crit_pnt2 = np.array([ - np.sqrt(c * (b - 1)), - np.sqrt(c * (b - 1)), b - 1])


#activation function
def act(x):
    return np.tanh(x)

#definition of the lorenz system
def lorenz(t, xyz, a, b, c):
    x = xyz
    dxdt = np.empty(3)
    dxdt[0] = a * (x[1] - x[0])
    dxdt[1] = x[0] * (b - x[2]) - x[1]
    dxdt[2] = x[0] * x[1] - c * x[2]
    return dxdt


def nonlin(x):
    return x / (1 + np.abs(x))

def test_func(x, b, normal, c):
    return 2 / np.pi * np.arctan(c * np.dot(normal, x - b))


def training_fun(reservoir, washout, steps, n, m, data, map, K, j, tr):
    for i in range(n * washout, n * steps):
        if i // n > j:
            j += 1
        k = i % n
        reservoir = np.append(reservoir, nonlin(K * reservoir[i - m] + data[:, j].T @ map[k, :]))
    if tr == False:
        reservoir = reservoir
    else:
        reservoir = np.reshape(reservoir[n * washout: n * steps], (steps - washout, n)).T
    return reservoir

def prediction_fun(n, m, steps, prediction, j, res, Wout_x, K, map):
    R = []
    reservoir = res
    d = Wout_x @ reservoir
    for i in range(n, n * prediction):
        if i // n >= j:
            j += 1
            d = Wout_x @ np.array(reservoir[-n: ])
            R.append(reservoir[-n: ])
            if j == 0 % 10:
                d = coo[:, j - 1]
        k = i % n
        #print(np.shape(reservoir))
        reservoir = np.append(reservoir, nonlin(K * reservoir[i - m] + d @ map[k, :]))
    R = np.array(R).T
    x = Wout_x @ R
    return x

In [20]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt


#reservoir size
n = 100

#timedomain
tmax = 22

#values of numbers of washout-, training -and prediction phase
washout = 500
training = 1000
prediction_time = 20 #time unit

#constant K
K = 0.1

# distance of steps we want to use from the past
m = n // 2

#lorenz system parameters
a, b, c = 10.0, 30.0, 2

#initial value
x0 = np.array([1.0, 0.0, 0.0])

# computation of stepsizes
steps = washout + training
prediction = int((steps / tmax) * prediction_time)

#timeaxis
t = np.linspace(0, tmax, steps)

#data creation
sol = solve_ivp(lorenz, (0, tmax), x0, method='RK45', t_eval=t, args=(a, b, c))
time = np.array(sol.t)
data = np.array(sol.y)
x, y, z = data

#extend solution
xnew = data[:, -1]

t = np.linspace(0, prediction_time, prediction + 1)
sol = solve_ivp(lorenz, (0, prediction_time), xnew, method='RK45', t_eval=t, args=(a, b, c))

time = np.array(sol.t)[1:]
coo = np.array(sol.y)[:, 1:]
c1, c2, c3 = coo


In [48]:
map = np.random.uniform(-0.05, 0.05, size=(n, 3))

#for i in range(n):
#    for j in range(3):
#        if np.random.random() < 2/3:
#            map[i][j] = 0

reservoir = np.random.uniform(0, 1, size=n)
reservoir = training_fun(reservoir, 1, washout, n, m, data, map, K, 1, False)
reservoir_state_matrix = training_fun(reservoir, washout, steps, n, m, data, map, K, washout, True)
target = data[:, washout:steps]

Wout_x = np.linalg.lstsq(reservoir_state_matrix.T, target.T, rcond=None)[0].T

# computation of the output matrix via Tikhonov regularization with regularization term beta (worse than numpy)
# beta = 1e-8
# Wout_x = target @ reservoir_state_matrix.T @ np.linalg.inv(reservoir_state_matrix @ reservoir_state_matrix.T + beta * np.identity(n))


print('norm = ', np.linalg.norm(Wout_x @ reservoir_state_matrix - target))

res_vec = reservoir_state_matrix.T[-1]

norm =  0.08091983454881503


In [59]:
norm = 101
count = 0

while norm > 100 and count < 100:
    count += 1
    x = prediction_fun(n, m, steps, prediction + 1, 1, res_vec, Wout_x, K, map)
    norm = max(np.linalg.norm(x, axis=0))
    print(count)

#plot of the lorenz attractor
linewidth = 0.8

fig = plt.figure('lorenz system prediction', figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')

ax.plot3D(*data, 'blue', linewidth=linewidth)
ax.plot(*crit_pnt1, 'fuchsia', marker='o', markersize=2)
ax.plot(*crit_pnt2, 'fuchsia', marker='o', markersize=2)

ax.set_title('solution')


#plot of the extended solution
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.set_title('prediction')
ax.plot3D(*coo, 'green', label='solve_ivp solution', linewidth=linewidth)
ax.plot(*coo[:, 0], 'red', marker='o', markersize=2)
ax.plot(*coo[:, -1], 'black', marker='o', markersize=2)


#plot of the lorenz system prediction
ax.plot3D(*x, 'violet', label='esn prediction', linewidth=linewidth)

ax.plot(*crit_pnt1, 'fuchsia', marker='o', markersize=2)
ax.plot(*crit_pnt2, 'fuchsia', marker='o', markersize=2)
ax.plot(*x[:, 0], 'red', marker='o', markersize=2)
ax.plot(*x[:, -1], 'black', marker='o', markersize=2)


plt.legend()
plt.show()

1
